### Binary classification with a Non-Deep learning algorithm 
We will use Support Vector machine with Spark


In [1]:
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install pyspark==2.4.5
!pip install -U scikit-learn

     |████████████████████████████████| 14.5MB 9.9MB/s eta 0:00:01
ERROR: tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
ERROR: autoai-libs 1.10.5 has requirement pandas>=0.24.2, but you'll have pandas 0.24.1 which is incompatible.
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
     |████████████████████████████████| 10.5MB 8.8MB/s eta 0:00:01
ERROR: ibm-watson-machine-learning 1.0.10 has requirement pandas<=0.25.3, but you'll have pandas 1.1.2 which is incompatible.
  Found existing installation: pandas 0.24.1
    Uninstalling pandas-0.24.1:
      Successfully uninstalled pandas-0.24.1
     |████████████████████████████████| 217.8MB 9.5MB/s eta 0:00:011     |██████████████████▏             | 123.3MB 42.5MB/s eta 0:00:03��██▉           | 141.8MB 42.5MB/s eta 0:00:02     |██████████████████████          | 149.1MB 51.3MB/s eta 0:00:02
     |████████████████████████████████| 204kB 50.6

In [2]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [3]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()


In [4]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from time import time
import logging
import numpy as np
import pandas as pd
import string
import re
from datetime import datetime
from packaging import version

from ibm_botocore.client import Config
import ibm_boto3

from sklearn.model_selection import train_test_split

In [5]:
# The code was removed by Watson Studio for sharing.

File Downloaded
File Downloaded


In [6]:
dfNewsTemp = pd.read_pickle('dfTrueFalseNews.pkl')
#dfNews['truthvalue'] = pd.Categorical(dfNews['truthvalue'])
x = dfNewsTemp['text'].values

y = dfNewsTemp['truthvalue'].values

print (dfNewsTemp.shape, dfNewsTemp.columns, '\n', dfNewsTemp.dtypes, type(x), type(y))


(1126, 3) Index(['text', 'source', 'truthvalue'], dtype='object') 
 text          object
source        object
truthvalue    object
dtype: object <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [7]:
l = dfNewsTemp.index.tolist()
d = {}
for i,v in enumerate(l):
    d[v]=i
#d['buzzfeed_line_20']
l[999:]

['buzzfeed_line_20',
 'buzzfeed_line_21',
 'buzzfeed_line_22',
 'buzzfeed_line_23',
 'buzzfeed_line_24',
 'buzzfeed_line_25',
 'buzzfeed_line_27',
 'buzzfeed_line_28',
 'buzzfeed_line_30',
 'buzzfeed_line_31',
 'buzzfeed_line_32',
 'snopes_line_54',
 'snopes_line_59',
 'snopes_line_65',
 'snopes_line_68',
 'snopes_line_69',
 'snopes_line_73',
 'snopes_line_75',
 'snopes_line_77',
 'snopes_line_82',
 'snopes_line_85',
 'snopes_line_88',
 'snopes_line_93',
 'snopes_line_101',
 'snopes_line_115',
 'snopes_line_121',
 'snopes_line_122',
 'snopes_line_123',
 'snopes_line_125',
 'snopes_line_126',
 'snopes_line_127',
 'snopes_line_129',
 'snopes_line_131',
 'snopes_line_132',
 'snopes_line_133',
 'snopes_line_134',
 'snopes_line_137',
 'snopes_line_140',
 'snopes_line_141',
 'snopes_line_142',
 'snopes_line_143',
 'snopes_line_145',
 'snopes_line_149',
 'snopes_line_150',
 'snopes_line_151',
 'snopes_line_152',
 'snopes_line_153',
 'snopes_line_154',
 'snopes_line_155',
 'snopes_line_156',
 

### Prepare the Text
1. Change all the text to lower case
2. Word Tokenization
3. Remove Stop words
4. Remove Non-alpha text
5. Word Lemmatization

In [8]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn
from collections import defaultdict
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# reproduce the same result every time the script is run.
np.random.seed(500)

[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/dsxuser/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [35]:
#dfNewsTemp.loc['snopes_line_311','text_final'] = " ".join(['face', 'dog', 'cat'])
#dfNewsTemp.astype({'text_final':'object'})
#dfNewsTemp.tail()
#dfNewsTemp.dtypes
#dfNewsTemp.loc[0,'text_value']=list([1,2,3])
#dfNewsTemp['text']
#ctr=0
# for index,entry in enumerate(dfNewsTemp['text']):
#     if ctr < 3: 
#         print (index, " ", entry)
#     ctr +=1
#dfNewsTemp['text_final'][0]
type(dfNewsTemp.loc['snopes_line_311','text_final'])

str

In [9]:
# Step 1: Change all the text to lower case.
dfNewsTemp['text'] = [entry.lower() for entry in dfNewsTemp['text']]

In [31]:
# Step 2: Tokenization : Each entry will be broken into set of words
dfNewsTemp['text']= [word_tokenize(entry) for entry in dfNewsTemp['text']]

In [32]:
# Step 3,4,5: Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [33]:
# Initializing WordNetLemmatizer()
word_Lemmatized = WordNetLemmatizer()
#dfNewsTemp.loc['text_final'].astype('object')
col = []
for index,entry in enumerate(dfNewsTemp['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    #word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        #if word not in stopwords.words('english') and word.isalpha():
        if word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    
    #dfNewsTemp.loc[index,'text_final'] = " ".join(Final_words)
    #dfNewsTemp.loc[index,'text_final'] = Final_words
    col.append(" ".join(Final_words))
dfNewsTemp['text_final'] = col


In [23]:
#dfNewsTemp['text_final'] = col
#col[-1]

In [36]:
# This took a while, so let's save the result.
dfNewsTemp.to_pickle('dfTrueFalseNews_tokenized.pkl', protocol=4 )

# save to our cloud storage 
upload_file_cos(credentials_news,'dfTrueFalseNews_tokenized.pkl','dfTrueFalseNews_tokenized.pkl')

 File Uploaded


In [10]:
dfTrueFalseNews_tokenized  = download_file_cos(credentials_news,'dfTrueFalseNews_tokenized.pkl', 'dfTrueFalseNews_tokenized.pkl')
!ls -al

File Downloaded
total 7228
drwxr-x--- 2 dsxuser dsxuser    4096 Sep 14 21:41 .
drwx------ 1 dsxuser dsxuser    4096 Sep 14 21:41 ..
-rw-r----- 1 dsxuser dsxuser 2228457 Sep 14 21:41 dfTrueFalseNews.pkl
-rw-r----- 1 dsxuser dsxuser 5159678 Sep 14 21:41 dfTrueFalseNews_tokenized.pkl


In [11]:
dfTrueFalseNews_tokenized = pd.read_pickle('dfTrueFalseNews_tokenized.pkl')
dfTrueFalseNews_tokenized.head()

,text,source,truthvalue,text_final
tech003legit,"[a, google, computer, victorious, over, the, w...",MihalceaNewsLegit,1,a google computer victorious over the world ch...
polit11legit,"[white, house, keeps, up, sanctuary, cities, p...",MihalceaNewsLegit,1,white house keep up sanctuary city pressure wi...
biz40legit,"[why, silicon, valley, is, n't, fighting, to, ...",MihalceaNewsLegit,1,why silicon valley be fight to save the intern...
edu10legit,"[protesters, disrupt, devos, school, visit, pr...",MihalceaNewsLegit,1,protester disrupt devos school visit protester...
tech038legit,"[solar-powered, 'skin, ', could, make, prosthe...",MihalceaNewsLegit,1,could make prosthetics more real many people t...


In [12]:
from sklearn import model_selection
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dfTrueFalseNews_tokenized['text_final'],dfTrueFalseNews_tokenized['truthvalue'],test_size=0.1)
#print(dfTrueFalseNews_tokenized.shape, (dfTrueFalseNews_tokenized['text_final'].shape), (dfTrueFalseNews_tokenized['truthvalue'].shape))
#dfTrueFalseNews_tokenized['text_final'][0]

In [13]:
#corpus = [" ".join(i) for i in dfTrueFalseNews_tokenized['text_final']]
#corpus[1:3]
dfTrueFalseNews_tokenized.head()

#dfTrueFalseNews_tokenized.dtypes
#import sklearn
#sklearn.show_versions()

,text,source,truthvalue,text_final
tech003legit,"[a, google, computer, victorious, over, the, w...",MihalceaNewsLegit,1,a google computer victorious over the world ch...
polit11legit,"[white, house, keeps, up, sanctuary, cities, p...",MihalceaNewsLegit,1,white house keep up sanctuary city pressure wi...
biz40legit,"[why, silicon, valley, is, n't, fighting, to, ...",MihalceaNewsLegit,1,why silicon valley be fight to save the intern...
edu10legit,"[protesters, disrupt, devos, school, visit, pr...",MihalceaNewsLegit,1,protester disrupt devos school visit protester...
tech038legit,"[solar-powered, 'skin, ', could, make, prosthe...",MihalceaNewsLegit,1,could make prosthetics more real many people t...


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer(max_features=26000)

Tfidf_vect.fit(dfTrueFalseNews_tokenized['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [15]:
print(len(Tfidf_vect.vocabulary_), Test_X_Tfidf)


16836   (0, 16781)	0.17118384142343043
  (0, 16639)	0.04185565787113178
  (0, 16600)	0.05954997292225774
  (0, 16589)	0.02301154925052861
  (0, 16567)	0.08131469889955778
  (0, 16538)	0.12265734559103758
  (0, 16529)	0.03399451527586988
  (0, 16348)	0.04530593722490369
  (0, 16327)	0.12265734559103758
  (0, 16326)	0.24531469118207516
  (0, 16310)	0.08738577707399033
  (0, 16259)	0.12265734559103758
  (0, 15919)	0.08998106727609378
  (0, 15783)	0.08311653634401397
  (0, 15485)	0.0504735652596911
  (0, 15455)	0.11038670810281244
  (0, 15357)	0.09965641456221547
  (0, 15216)	0.09066031866794538
  (0, 15209)	0.08075265842869195
  (0, 15099)	0.058575261367563684
  (0, 15059)	0.03146386007079189
  (0, 15015)	0.1603177735792545
  (0, 15012)	0.021586417845775294
  (0, 15004)	0.04339600181875995
  (0, 14908)	0.03564632225517207
  :	:
  (112, 2558)	0.06389203383775867
  (112, 2260)	0.04005697123346543
  (112, 2259)	0.04162023946962601
  (112, 2116)	0.018574914916278124
  (112, 2097)	0.0180113323

### Use Naive Bayes Classifier

In [16]:
# some open source utilities to evaluate a model.
# you can see them at: https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/notebooks/Ch05_Building_Tuning_and_Deploying_Models/model_evaluation_utils.py
download_file_cos(credentials_news,'model_evaluation_utils.py', 'model_evaluation_utils.py')
import importlib
#importlib.reload(model_evaluation_utils)
import model_evaluation_utils as meu

File Downloaded


In [20]:
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Train_y_int = Train_Y.astype('int')
Test_y_int = Test_Y.astype('int')
Naive.fit(Train_X_Tfidf,Train_y_int)# predict the labels on validation dataset
#Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_y_int)* 100)
print("Naive model params: {}".format(Naive.get_params()))
meu.display_model_performance_metrics(true_labels=Test_Y.tolist(), predicted_labels=predictions_NB.tolist())

Naive Bayes Accuracy Score ->  36.283185840707965
Naive model params: {'alpha': 1.0, 'class_prior': None, 'fit_prior': True}
Model Performance metrics:
------------------------------
Accuracy: 0.3628
Precision: 0.3381
Recall: 0.3628
F1 Score: 0.3293

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           1       0.29      0.15      0.20        59
           0       0.39      0.59      0.47        54

    accuracy                           0.36       113
   macro avg       0.34      0.37      0.34       113
weighted avg       0.34      0.36      0.33       113


Prediction Confusion Matrix:
------------------------------
          Predicted:    
                   1   0
Actual: 1          9  50
        0         22  32


## Use Support Vector Machine

In [19]:
from sklearn import svm
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=2.5, kernel='sigmoid', degree=3, gamma='scale')
SVM.fit(Train_X_Tfidf,Train_y_int)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> {}".format(accuracy_score(predictions_SVM, Test_y_int)*100))
print("SVM model params: {}".format(Naive.get_params()))
meu.display_model_performance_metrics(true_labels=Test_Y.tolist(), predicted_labels=predictions_SVM.tolist())

SVM Accuracy Score -> 59.29203539823009
SVM model params: {'alpha': 1.0, 'class_prior': None, 'fit_prior': True}
Model Performance metrics:
------------------------------
Accuracy: 0.5929
Precision: 0.6035
Recall: 0.5929
F1 Score: 0.5889

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           1       0.64      0.49      0.56        59
           0       0.56      0.70      0.62        54

    accuracy                           0.59       113
   macro avg       0.60      0.60      0.59       113
weighted avg       0.60      0.59      0.59       113


Prediction Confusion Matrix:
------------------------------
          Predicted:    
                   1   0
Actual: 1         29  30
        0         16  38


### Split the data

In [ ]:
# use the dataframe to write a parquet file
dfNewsTemp.to_parquet('dfTrueFalseNews_tokenized.parquet')
dfNews = spark.read.parquet('dfTrueFalseNews_tokenized.parquet')
dfNews.createOrReplaceTempView('dfNews')

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dfNewsTemp['text_final'],Corpus['label'],test_size=0.1)

In [ ]:
### Prepare the data for Spark

In [ ]:
splits = dfNews.randomSplit([0.9, 0.1])
df_train = splits[0]
df_test = splits[1]



In [ ]:
dfNews.show()

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler

indexer = StringIndexer(inputCol="text", outputCol="text_index")
encoder = OneHotEncoder(inputCol="text_index", outputCol="textVec")
vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")